In [3]:
# Uygulama 4.4
from random import choice
from experta import *

class Doctor(Fact):
    """   """
    pass

class goToDoctor(KnowledgeEngine):
    # dis fircalarken dis eti kanamasi olursa
    @Rule(Doctor(issue="short bleeding"))
    def tissueBleeding(self):
        print("dis hastaligi vardir ve dis hekimine basvur")

    # dis fircalarken uzun sureli dis eti kanamasi olursa
    @Rule(Doctor(issue="too much bleeding"))
    def tooMuchTissueBleeding(self):
        print("diseti cekilmesi vardir ve dis hekimine basvur")

    # eger dis eti cekilmesi var ve dis koku gorunuyorsa
    @Rule(Doctor(issue="teeth root visible"))
    def isTeethRootVisible(self):
        print("dolgu yaptir")

    # diste yiyecek ve iceceklerden olusan renk degisimi varsa
    @Rule(Doctor(issue="color change"))
    def cleanTeeth(self):
        print("disleri temizle")

    # yeni dis cikarirken morarma gorunuyorsa
    @Rule(Doctor(issue="it's purple"))
    def isItPurple(self):
        print("dis hekimine basvur")

    # diste agri yapmayan curuk varsa
    @Rule(Doctor(issue="rotten teeth"))
    def fillUp(self):
        print("dolgu yaptir")

    # disteki curuk ileri derecedeyse
    @Rule(Doctor(issue="severe rotten teeth"))
    def canalCure(self):
        print("kanal tedavisi ve dolgu yaptir")


uzman = goToDoctor()

uzman.reset()
uzman.declare(Doctor(issue=choice(['short bleeding', 'too much bleeding', 'teeth root visible',
                                   'color change', 'it\'s purple', 'rotten teeth', 'severe rotten teeth'])))
uzman.run()


disleri temizle


In [ ]:
# Uygulama 5.4
# Yaprak siniflandirmasi (1DESA)

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

# test ve egitim verilerinin okunmasi
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Siniflarin belirlenmesi ve etiketlenmesi
label_encoder = LabelEncoder().fit(train.species)
labels = label_encoder.transform(train.species)
classes = list(label_encoder.classes_)

# verilerin hazirlanmasi, ozellik ve sinif sayisinin belirlenmesi
train = train.drop(["id", "species"], axis=1)
test = test.drop(["id"], axis=1)
nb_features = 192
nb_classes = len(classes)

# egitim verisindeki verilen standartlastirilirmasi
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(train.values)
train = scaler.transform(train.values)

# egitim verisinin egitim ve dogrulama icin ayarlanmasi
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, labels, test_size=0.1)

# etiketlerin kategorilerinin belirlenmesi
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

# giris verilerinin boyutlarinin ayarlanmasi
x_train = np.array(x_train).reshape(891, 192, 1)
x_valid = np.array(x_valid).reshape(99, 192, 1)

# 1DESA modelinin olusturulmasi
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(512, 1, input_shape=(nb_features, 1)))
model.add(Activation("relu"))
model.add(MaxPooling1D(2))
model.add(Conv1D(256, 1))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2048, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(nb_classes, activation="softmax"))
model.summary()

# agin derlenmesi
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# modelin egitilmesi
model.fit(x_train, y_train, epochs=10, validation_data=(x_valid, y_valid))

# ortalama degerlerin gosterilmesi
print("Ortalama Egitim Kaybi: ", np.mean(model.history.history["loss"]))
print("Ortalama Egitim Basarimi: ", np.mean(model.history.history["accuracy"]))
print("Ortalama Dogrulama Kaybi: ", np.mean(model.history.history["val_loss"]))
print("Ortalama Dogrulama Basarim: ", np.mean(model.history.history["val_accuracy"]))

dtc = DecisionTreeClassifier()
y_pred = dtc.predict(x_valid)
score = f1_score(y_pred, y_valid)
print("F1 score: {:.2f}".format(score))

In [ ]:
# Uygulama 6.4
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from sklearn.preprocessing import LabelEncoder

# test ve egitim verilerinin okunmasi
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Siniflarin belirlenmesi ve etiketlenmesi
label_encoder = LabelEncoder().fit(train.species)
labels = label_encoder.transform(train.species)
classes = list(label_encoder.classes_)

train = train.drop(["id"], axis=1)
test = test.drop(["id"], axis=1)
nb_features = 192
nb_classes = len(classes)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(train.values)
train = scaler.transform(train.values)

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, labels, test_size=0.1)

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

x_train = np.array(x_train).reshape(891, 192, 1)
x_valid = np.array(x_valid).reshape(99, 192, 1)

IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3

filenames = os.listdir("CNN/train")
categories = []
for filename in filenames:
	category = filename.split('.')[0]
	if category == 'tumor':  
		categories.append(1)
	else:
		categories.append(0)  
df = pd.DataFrame({'filename': filenames, 'category': categories})

df['category'].value_counts().plot.bar()

# modelin olusturulmasi
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))  # nb_classes tane sinif
model.summary()

#  modelin derlenmesi
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# verinin hazirlanmasi
train_df, validate_df = train_test_split(df, test_size= 0.2)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
# kategorilere bakilmasi
train_df["category"].value_counts().plot.bar()
# egitim ve dogrulama verisinin hazirlanmasi
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size = 15

#  egitim verilerinin cogaltilmasi
train_datagen = ImageDataGenerator(
	rotation_range = 15,
	rescale = 1./255,
	shear_range = 0.1,
	zoom_range = 0.2,
	horizontal_flip = True,
	width_shift_range = 0.1,
	height_shift_range = 0.1
)

train_generator = train_datagen.flow_from_dataframe(
	train_df,
	"images",
	x_col = 'filename',
	y_col = 'category',
	target_size=IMAGE_SIZE,
	class_mode='categorical',
	batch_size=batch_size
)

# dogrulama verilerinin cogaltilmasi
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator =validation_datagen.flow_from_dataframe(
	validate_df,
	"images",
	x_col = 'filename',
	y_col = 'category',
	target_size=IMAGE_SIZE,
	class_mode='categorical',
	batch_size=batch_size
)

# modelin egitilmesi
epochs = 15  # 10
history = model.fit(
	train_generator,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=total_validate//batch_size,
	steps_per_epoch=total_train//batch_size
)

# olusturulan modelin kaydedilmesi
model.save_weights("model1.h5")
# tahmin isleminin yapilmasi
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [ ]:
# Uygulama 7.4
import numpy as np  # linear algebra
import pandas as pd  # CSV file
import scipy.io.wavfile as sci_wav  # Open wav files
import matplotlib.pyplot as plt
import numpy as np
import random

ROOT_DIR = "CNN Voice/cats_dogs/"
CSV_PATH = "CNN Voice/train_test_split.csv"


def read_wav_files(wav_files):
	if not isinstance(wav_files, list):
		wav_files = [wav_files]
	return [sci_wav.read(ROOT_DIR + f)[1] for f in wav_files]


def get_trunk(_X, idx, sample_len, rand_offset=False):
	randint = np.random.randint(10000) if rand_offset is True else 0
	start_idx = (idx * sample_len + randint) % len(_X)
	end_idx = ((idx + 1) * sample_len + randint) % len(_X)
	if end_idx > start_idx:  # normal case
		return _X[start_idx: end_idx]
	else:
		return np.concatenate((_X[start_idx:], _X[:end_idx]))


def get_augmented_trunk(_X, idx, sample_len, added_samples=0):
	X = get_trunk(_X, idx, sample_len)

	# Add other audio of the same class to this sample
	for _ in range(added_samples):
		ridx = np.random.randint(len(_X))  # random index
		X = X + get_trunk(_X, ridx, sample_len)

	# One might add more processing (like adding noise)

	return X


def dataset_gen(is_train=True, batch_shape=(20, 16000), sample_augmentation=0):
	s_per_batch = batch_shape[0]
	s_len = batch_shape[1]

	X_cat = dataset['train_cat'] if is_train else dataset['test_cat']
	X_dog = dataset['train_dog'] if is_train else dataset['test_dog']

	# Go through all the permutations
	y_batch = np.zeros(s_per_batch)
	X_batch = np.zeros(batch_shape)
	# Random permutations (for X indexes)
	nbatch = int(max(len(X_cat), len(X_cat)) / s_len)
	perms = [list(enumerate([i] * nbatch)) for i in range(2)]
	perms = sum(perms, [])
	random.shuffle(perms)



	while len(perms) > s_per_batch:

		# Generate a batch
		for bidx in range(s_per_batch):
			perm, _y = perms.pop()  # Load the permutation
			y_batch[bidx] = _y

			# Select wether the sample is a cat or a dog
			_X = X_cat if _y == 0 else X_dog

			# Apply the permutation to the good set
			if is_train:
				X_batch[bidx] = get_augmented_trunk(
					_X,
					idx=perm,
					sample_len=s_len,
					added_samples=sample_augmentation)
			else:
				X_batch[bidx] = get_trunk(_X, perm, s_len)

		yield (X_batch.reshape(s_per_batch, s_len, 1),
		       y_batch.reshape(-1, 1))


def load_dataset(dataframe):
	df = dataframe

	dataset = {}
	for k in ['train_cat', 'train_dog', 'test_cat', 'test_dog']:
		v = list(df[k].dropna())
		v = read_wav_files(v)
		v = np.concatenate(v).astype('float32')

		# Compute mean and variance
		if k == 'train_cat':
			dog_std = dog_mean = 0
			cat_std, cat_mean = v.std(), v.mean()
		elif k == 'train_dog':
			dog_std, dog_mean = v.std(), v.mean()

		# Mean and variance suppression
		std, mean = (cat_std, cat_mean) if 'cat' in k else (dog_std, dog_mean)
		v = (v - mean) / std
		dataset[k] = v

		print('loaded {} with {} sec of audio'.format(k, len(v) / 16000))

	return dataset


df = pd.read_csv(CSV_PATH)
dataset = load_dataset(df)

batch_size = 512
num_data_points = 16000
n_augment = 10

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, BatchNormalization
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv1D(20, 4, strides=2, activation='relu', input_shape=(num_data_points, 1)))
model.add(BatchNormalization())
model.add(Conv1D(20, 4, strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(40, 4, strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(40, 4, strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Conv1D(80, 4, strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(80, 4, strides=2, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

NUM_EPOCHS = 50
adam_optimizer = Adam(decay=1e-3)
model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

NUM_EPOCHS = 50
train_loss = []
val_loss = []
train_acc = []
val_acc = []

# loop through epoch samples (batchs)
for epochs in range(NUM_EPOCHS):
	train_gen = dataset_gen(is_train=True, batch_shape=(batch_size, num_data_points), sample_augmentation=n_augment)
	for batch_x, batch_y in train_gen:
		history = model.fit(batch_x, batch_y, epochs=1, validation_split=0.2)
		train_loss.extend(history.history['loss'])
		val_loss.extend(history.history['val_loss'])
		train_acc.extend(history.history['accuracy'])
		val_acc.extend(history.history['val_accuracy'])

fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(111)
ax.plot(train_loss, label='train loss')
ax.plot(val_loss, label='val loss', color='green')
plt.legend()
plt.title('log loss')
plt.show()

fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(111)
ax.plot(train_acc, label='train accuracy')
ax.plot(val_loss, label='val accuracy', color='green')
plt.legend()
plt.title('accuracy')
plt.show()